In [1]:
import sys
sys.path.insert(0, '../src/libs')

In [23]:
# our file imports
from music_transformer import *
from transformer_training_helpers import *
from song_classification import *
from dataset import *
from midi2audio import FluidSynth

# musicautobot
from musicautobot.numpy_encode import *
from musicautobot.config import *
from musicautobot.music_transformer import *
from musicautobot.utils.midifile import *
from musicautobot.utils.file_processing import process_all

In [3]:
def topk_decode(model, src, src_mask, max_len, start_symbol, k):
  memory = model.encode(src, src_mask)
  ys = torch.ones(1, 1).fill_(start_symbol).type_as(src.data)
  for i in range(max_len-1):
    out, w2 = model.decode(memory, src_mask,
                       Variable(ys),
                       Variable(subsequent_mask(ys.size(1))
                                .type_as(src.data)))
    prob = model.generator(out[:, -1])
    _, next_words = torch.topk(prob, k = k, dim = 1)
    # if starting, then create 
    #print(next_words)
    if i == 0:
      next_word = next_words[0][0].data.item()
    else:
      k2 = random.randint(0,k-1)
      next_word = next_words[0][k2].data.item()
    #print(next_word)
    #k = random.randint(0,4)
    #next_word = next_words[k].data[0]
    ys = torch.cat([ys, torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)
  
  return ys

def greedy_decode(model, src, src_mask, max_len, start_symbol):
    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type_as(src.data)
    for i in range(max_len-1):
        out, w2 = model.decode(memory, src_mask, 
                           Variable(ys), 
                           Variable(subsequent_mask(ys.size(1))
                                    .type_as(src.data)))
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim = 1) # change this for top k
        next_word = next_word.data[0]
        ys = torch.cat([ys, 
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)
    return ys

Load model from previous state

In [17]:
myModel = torch.load('../models/new_dataset_epochs_30_LM.pt')

Load data

In [6]:
myDs = MidiDataset('../numpy_path', 150, 'N/A', '.npy')
dataloader = DataLoader(myDs, 1, shuffle=True)

for _, item in enumerate(dataloader):
    print(item)
    break

tensor([[[  0.,   1.,  81.,  ..., 139.,   8., 139.],
         [ 52., 139.,  52.,  ..., 143.,  45., 143.],
         [  8., 141.,  71.,  ..., 139.,  88., 142.],
         ...,
         [ 74., 138.,  54.,  ..., 139.,  85., 140.],
         [ 50., 139.,  50.,  ..., 143.,  47., 143.],
         [  8., 141.,  81.,  ..., 139.,  54., 139.]]])


In [7]:
# data_path = './saved_data/'
# data_save_name = 'music_item_data_0-10000.pkl'
# numpy_path = '../numpy_path'

# numpy_files = get_files(numpy_path, extensions='.npy', recurse=True); len(numpy_files)

# def create_databunch(files, path):
#     #save_file.parent.mkdir(exist_ok=True, parents=True)
#     vocab = MusicVocab.create()
#     processors = [OpenNPFileProcessor(), MusicItemProcessor()]

#     #data = MusicDataBunch.from_files(midi_files, path, processors=processors, encode_position=True)
#     data = MusicDataBunch.from_files(files, path, processors=processors, encode_position=True)
#     return data

# all_data = create_databunch(numpy_files, data_path); all_data

In [14]:
print(myDs[0])
new_tens = myDs[0].reshape(myDs[0].shape[0] * myDs[0].shape[1])
print(new_tens.shape)
print(new_tens)

tensor([[  0.,   1.,   8.,  ..., 141.,  44., 145.],
        [  8., 140.,  56.,  ..., 138.,  75., 140.],
        [ 51., 141.,  39.,  ..., 157.,  51., 145.],
        ...,
        [ 70., 138.,   8.,  ..., 140.,  78., 138.],
        [ 70., 138.,  66.,  ..., 145.,   8., 141.],
        [ 47., 141.,   8.,  ..., 168.,  70., 168.]])
torch.Size([7800])
tensor([  0.,   1.,   8.,  ..., 168.,  70., 168.])


Generate topk/greedy outputs from the model

In [33]:
myModel.eval()
# ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
#src = Variable(torch.LongTensor([[1,2,3,4,5,6,7,8,9,10]]))
#src = Variable(torch.LongTensor([[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26]])).cuda()
idx = 1
src = myDs[idx].reshape(myDs[idx].shape[0] * myDs[idx].shape[1])[:100].unsqueeze(dim=0).long().cuda() # all_data.valid_ds[1][0].to_tensor()[:100].unsqueeze(dim=0)
#print(src.size())
src_mask = Variable(torch.ones(1, 1, 100)).unsqueeze(dim=0).cuda() # 26 was 10
output = greedy_decode(myModel.cuda(), src, src_mask, max_len=150, start_symbol=0)
output2 = topk_decode(myModel.cuda(), src, src_mask, max_len=150, start_symbol=0, k=2)

#print(output2[0])
#print(output[0])

output.cuda()
output2.cuda()

print("GREEDY: ", output[0])
print("TOPK: ", output2[0])
print("SRC: ", src[0])

ncorrect = 0
for i in range(len(src)):
  if src[0][i] == output2[0][i]:
    ncorrect += 1
print("TOPK: ", ncorrect / len(src[0]))

ncorrect = 0
for i in range(len(src)):
  if src[0][i] == output[0][i]:
    ncorrect += 1
print("GREEDY: ", ncorrect / len(src[0]))

GREEDY:  tensor([  0, 139, 146, 159, 109, 153,  52, 167, 176,  45, 185, 184, 200,  68,
        169,  59, 163,  52, 138,  45,  67, 141,  59, 233,   8, 143,  63, 139,
         52, 185,  35, 163, 142,  40, 143,  56,  86,  56,  45, 102,  45, 170,
         40,  64,  59, 157,  45, 147,   8, 147,  45, 147,  45,  43, 139,   8,
        147,  59, 233,  35,  35, 151,   8, 138,  39, 146,   8,  56, 146,  49,
        152,  37, 232,   8, 160,  85, 143,  78, 152,  75, 152,  54, 152,  54,
         63, 160,   8, 139,  83, 152, 154,  34,  33,  80,  64, 152,  73, 141,
         73, 185, 185,  59,  59,  59,  76, 141,  76, 164,  59, 157,  47, 163,
         52, 138,  40, 141,   8, 139,  71, 138,  63, 202, 143, 163, 143,   8,
        169,  99, 172, 102, 163,  90,  43, 142,   8, 139,  37, 141,   8, 139,
         59,  60, 169,  35, 143,   8, 167,  39, 172,   8], device='cuda:0')
TOPK:  tensor([  0, 139, 146, 152,  68, 171,  53, 159, 184,  45, 185,  80, 177,  66,
        157, 143,  52, 139,  52, 138,   8, 157,  7

Convert outputs to .mid

In [51]:
%matplotlib inline
from music21 import *
import fluidsynth
#environment.set()
#print(output)
output[0, 1] = 1
output2[0, 1] = 1
#print(src.cpu().numpy())
temp_vocab = MusicVocab.create()
#print(temp_vocab.stoi)
MI = temp_vocab.to_music_item(src.cpu().numpy()[0])
MO = temp_vocab.to_music_item(output.cpu().numpy()[0])
MT = temp_vocab.to_music_item(output2.cpu().numpy()[0])
print("SRC.SHP: ", src.shape)
print("OUTPUT.SHP: ", output.shape)
print("OUTPUT2.SHP: ", output2.shape)
print(src)
print(output)
print(output2)
print(MI.to_text())
#print(MO.to_text())

MI.stream.write('midi', fp='./temp_outputs/inp.mid')
MO.stream.write('midi', fp='./temp_outputs/pred.mid')
MT.stream.write('midi', fp='./temp_outputs/topk.mid')

#path = os.path.join('./','inp.mid')
#print_play_mid('./inp.mid')
#print_mid('./inp.mid')
#print_mid('./pred.mid')
#print_mid('./topk.mid')

SRC.SHP:  torch.Size([1, 100])
OUTPUT.SHP:  torch.Size([1, 150])
OUTPUT2.SHP:  torch.Size([1, 150])
tensor([[  0,   1,  63, 139,  59, 139,  47, 138,   8, 138,  66, 138,   8, 138,
          71, 139,  66, 139,  47, 138,  47, 138,   8, 138,  75, 139,   8, 138,
          66, 138,  63, 139,  59, 139,   8, 139,  71, 139,  66, 139,  42, 139,
          42, 139,   8, 138,  75, 139,   8, 138,  66, 138,  63, 139,  59, 139,
          47, 138,  47, 138,   8, 139,  71, 139,  66, 139,  47, 139,  47, 138,
           8, 138,  75, 139,   8, 138,  66, 138,  63, 139,  59, 139,   8, 139,
          75, 139,  71, 139,  66, 139,  42, 139,   8, 139,  68, 139,  64, 139,
          59, 139]], device='cuda:0')
tensor([[  0,   1, 146, 159, 109, 153,  52, 167, 176,  45, 185, 184, 200,  68,
         169,  59, 163,  52, 138,  45,  67, 141,  59, 233,   8, 143,  63, 139,
          52, 185,  35, 163, 142,  40, 143,  56,  86,  56,  45, 102,  45, 170,
          40,  64,  59, 157,  45, 147,   8, 147,  45, 147,  45,  43, 139

IndexError: index 137 is out of bounds for axis 2 with size 128

Classify generated song's musical key

In [49]:
src_mitem = MusicItem.from_file('./temp_outputs/inp.mid', MusicVocab.create())
m_item = MusicItem.from_file('./temp_outputs/topk.mid', MusicVocab.create())
m_item2 = MusicItem.from_file('./temp_outputs/pred.mid', MusicVocab.create())
print(src_mitem.to_text())
print(int_to_note(34))
print(calculate_score(src_mitem))
print(calculate_score(m_item))
print(calculate_score(m_item2))

FileNotFoundError: Cannot find file in ./temp_outputs/topk.mid

Convert .mid to .wav

In [50]:
filepath = './temp_outputs/inp.mid'
fs = FluidSynth(sound_font='./FluidR3_GM.sf2')
fs.midi_to_audio(filepath, '{}.wav'.format(filepath))